In [ ]:
try:
    import cirq
    import cirq_google
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq-google --pre
    print("installed cirq.")
    import cirq
    import cirq_google

try:
    import qsimcirq
except ImportError:
    print("installing qsimcirq...")
    !pip install qsimcirq
    print("installed qsimcirq.")

import copy
import random
import time

In [ ]:
def rand2d(rows, cols):
    return[[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    h = rand2d(length, length)
    jr = rand2d(length, length)
    jc = rand2d(length, length-1)
    return (h, jr, jc)

h, jr, jc= random_instance(3)

In [ ]:
def prepare_plus_layer(length):
    for i in range(length):
        for j in range(length):
            yield cirq.H(cirq.GridQubit(i, j))

def rot_x_layer(length, half_turns):
    # define circuit once and reuse for each operation
    rot = cirq.XPowGate(exponent=half_turns)
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

def rot_z_layer(h, half_turns):
    gate = cirq.ZPowGate(exponent=half_turns)
    for i,h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
             yield gate(cirq.GridQubit(i, j))

def rot_11_layer(jr, jc, half_turns):
    cz_gate = cirq.CZPowGate(exponent=half_turns)
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i+1, j)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i, j+1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

In [ ]:
length = 5
qubits = cirq.GridQubit.square(length)
h, jr, jc = random_instance(length)

circuit = cirq.Circuit()
circuit.append(prepare_plus_layer(length))
circuit.append(rot_z_layer(h, 0.3))
circuit.append(rot_11_layer(jr, jc, 0.3))
circuit.append(rot_x_layer(length, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))

In [ ]:
simulator = cirq.Simulator()

ngpus = 1
qsim_options = qsimcirq.QSimOptions(
    max_fused_gate_size = 2,
    cpu_threads = 1,
    gpu_mode = ngpus,
    use_gpu = True,
)

# simulator_cuQ = qsimcirq.QSimSimulator(qsim_options)

In [ ]:
start = time.time()
results = simulator.run(circuit, repetitions = 100)
print(results.histogram(key='x'))
print("CPU runtime: {: .2f}s".format(time.time() - start))